# Libraries

In [1]:
using Compat
using BenchmarkTools
using Plots
using LaTeXStrings
using Printf
using ODEInterface,ODEInterfaceDiffEq

include("ODE_Isotropic_Gas.jl");

In [4]:
##### Angular Distributions ######
function angular_dist_Ian(theta,a,b)
    #Neutrino Flavor Pendulum Reloaded: The Case of Fast Pairwise Conversion - Padilla-Gay, Irene Tamborra, Georg G. Raffelt 
    # https://arxiv.org/abs/2109.14627
    return 0.45 - a +(0.1/b)*exp(-1*(1-cos(theta))^2/(2*b^2))
end

function theta_dist(theta,nu_type,case)
    if nu_type=="nu_x" || nu_type=="nu_x_bar"
        return 0
    end
    
    # New Developments in Flavor Evolution of a Dense Neutrino Gas - Irene Tamborra, Shashank Shalgar
    #https://arxiv.org/abs/2011.01948
    if case==1
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            if theta<pi/3
                return 1.0
            else
                return 0.25
            end
        end
        
    elseif case==2
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            if theta<pi/4
                return 1.0
            else
                return 0.5
            end
        end
        
    elseif case==3
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            if theta<pi/3
                return 0.5*1.5
            else
                return 0.5
            end
        end
        
    #Neutrino Flavor Pendulum Reloaded: The Case of Fast Pairwise Conversion - Padilla-Gay, Irene Tamborra, Georg G. Raffelt 
    # https://arxiv.org/abs/2109.14627
    elseif case=='A'
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            return angular_dist_Ian(theta,0,0.4)
        end
    elseif case=='B'
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            return angular_dist_Ian(theta,0.02,0.4)  
        end
    elseif case=='C'
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            return angular_dist_Ian(theta,0.02,0.6)
        end
    elseif case=='D'
        if nu_type=="nu_e"
            return 0.5
        end
        if nu_type=="nu_e_bar"
            return angular_dist_Ian(theta,0.06,0.2)
        end
    else
        println("Not a valid angular distribution!")
    end
end;

In [25]:
function from_1D_to_MultiD_Theta(y,n_f,n_dim,n_theta)
    nu, nubar = [],[]
    num_diff_nu_compnents=2*n_f*n_dim
    #Filling [Energy bin][Nu_types][3components]
    for i in 1:n_theta
        push!(nu,[])
        push!(nubar,[])
        for j in 1:n_f
            push!(nu[i],[])
            push!(nubar[i],[])
            for k in 1:n_dim
                #nu 
                nu_index=((i-1)*num_diff_nu_compnents)+k+2*(j-1)*n_dim
                push!(nu[i][j],y[nu_index])
                #nubar   
                nubar_index=((i-1)*num_diff_nu_compnents)+(k+n_dim)+2*(j-1)*n_dim
                push!(nubar[i][j],y[nubar_index])
            end
        end
    end
    return nu, nubar
end;

function from_1D_to_MultiD_Energy_v2(sol,n_f,n_dim,n_E)
    nu, nubar = zeros(n_E,n_f,n_dim,length(sol.t)),zeros(n_E,n_f,n_dim,length(sol.t))
    num_diff_nu_compnents=2*n_f*n_dim
    #Filling [Energy bin][Nu_types][3components]
    for i in 1:n_E, j in 1:n_f, k in 1:n_dim
        #nu 
        nu_index=((i-1)*num_diff_nu_compnents)+k+2*(j-1)*n_dim
        nu[i,j,k,:]=sol[nu_index,:]
        #nubar   
        nubar_index=((i-1)*num_diff_nu_compnents)+(k+n_dim)+2*(j-1)*n_dim
        nubar[i,j,k,:]=sol[nubar_index,:]
    end
    return nu, nubar
end;

In [27]:
function func_FFC_uniform!(dy,y, params, time)
    omega,E,theta_V,mu_0,n_f,n_dim,n_theta,cos_theta_vec= params  # unpack parameters
    B=B_vec(n_dim,theta_V)
    L=L_vec(n_dim)
    mu=mu_0
    lamb=lambda_supernova(time,"no",0)

    derivs=[]
    nu, nubar = from_1D_to_MultiD_Theta(y,n_f,n_dim,n_theta)
    #Summed nu and nubar components
    nu_sum=sum(sum(nu))
    nubar_sum=sum(sum(nubar))
    nu_sum_theta=sum(sum(cos_theta_vec.*nu))
    nubar_sum_theta=sum(sum(cos_theta_vec.*nubar))
    
    num_diff_nu_compnents=2*n_f*n_dim
    # list of dy/dt=f functions
    for i in 1:n_theta, j in 1:n_f
        #nu 
        aux=B*omega+L*lamb-mu*((nu_sum-nubar_sum)-cos_theta_vec[i]*(nu_sum_theta-nubar_sum_theta))
        P_aux= cross_prod(nubar[i][j],aux)
        for k in 1:n_dim
            nu_index=((i-1)*num_diff_nu_compnents)+k+2*(j-1)*n_dim
            dy[nu_index]=P_aux[k]
        end
        #nubar
        
        aux=-1*B*omega+L*lamb-mu*((nu_sum-nubar_sum)-cos_theta_vec[i]*(nu_sum_theta-nubar_sum_theta))
        P_aux= cross_prod(nubar[i][j],aux)
        for k in 1:n_dim
            nubar_index=((i-1)*num_diff_nu_compnents)+(k+n_dim)+2*(j-1)*n_dim
            dy[nubar_index]=P_aux[k]
        end
    end

end;

In [30]:
#Neutrino Mixing
delta_m2=delta_m2_31
theta_V=10^-6 #rad
nu_types=["nu_e","nu_x"]
scenarios=[["NH",1,10,DP8()]]

mu_0=10^3 #km⁻¹
mu_s=mu_0*10^(-3)*c_const #1/s
println("mu =",mu_s," 1/s")
mu_0=mu_0/from_eV_to_1_over_km #eV
println("mu =",mu_0," eV")
seed=10^-7

#Numerical Solver Info
t_i,t_f=0,10*10^(-6) #s
println("t_f =",t_f," s")
theta_bins=100
cos_theta_step=2/theta_bins;

atol,rtol=8e-10,8e-7
alg_hints=[:auto];

mu =2.99792458e8 1/s
mu =1.9732698044404105e-7 eV
t_f =9.999999999999999e-6 s


In [31]:
mass_ord,case,E_nu,solver= scenarios[1]
println(mass_ord,", ",case,", ",E_nu,", ",solver)
y0,omega,E,t_i,t_f,mu_0,n_f,n_dim,n_theta,cos_theta_vec=initiate_FFC_uniform(nu_types,t_i,t_f,E_nu,delta_m2,mu_0,theta_bins,case,seed=seed)
### seems ok until here
if mass_ord=="NH" 
    params=omega,E,theta_V,mu_0,n_f,n_dim,n_theta,cos_theta_vec
elseif mass_ord=="IH"
    params=-1*omega,E,theta_V,mu_0,n_f,n_dim,n_theta,cos_theta_vec
else
    println("Not a mass ordering option!")
end
### seems ok until here
# println(params)
prob = ODEProblem(func_FFC_uniform!,y0,(t_i,t_f),params)
sol = solve(prob,alg=solver,alg_hints=alg_hints, reltol=rtol, abstol=atol)

NH, 1, 10, DP8()


retcode: Success
Interpolation: specialized 7th order interpolation
t: 224-element Vector{Float64}:
     0.0
     9.999999999999999e-5
     0.0006999999999999999
     0.0043
     0.0259
     0.15550000000000003
     0.9331000000000003
     5.598700000000002
    33.59230000000001
   201.55390000000006
  1209.3235000000004
  7255.941100000004
 43535.64670000003
     ⋮
     1.444328765034631e10
     1.4513288968023066e10
     1.4584350442629631e10
     1.4658452537943672e10
     1.4734868903976402e10
     1.4810365142047922e10
     1.4882025078846518e10
     1.495179610921563e10
     1.5024080704657679e10
     1.5099309025051495e10
     1.5175822507921843e10
     1.5192674479961273e10
u: 224-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.01, 0.0, 0.0, 0.005, 0.0, 0.0, -0.0, 0.0  …  0.01, 0.0, 0.0, 0.02, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0]
 [-2.662011809967529e-33, -1.249999999999166e-22, 0.01, -2.663574309967525e-33, 1.249999999999166e-22, 0.005, 0.0, 0.0, 0.0, 0.0  …  0.01, -7.85135717674635